In [1]:
%matplotlib inline

## Make cut on subhalo mass at $M_{\rm peak} >= 10^{11.25}M_{\odot}$

In [4]:
from halotools.sim_manager import CachedHaloCatalog, UserSuppliedHaloCatalog
halocat = CachedHaloCatalog(simname='bolplanck')

metadata = dict(Lbox=halocat.Lbox, 
               particle_mass=halocat.particle_mass, 
               redshift=halocat.redshift)

subhalo_mpeak_cut = 10**11.25

mask = halocat.halo_table['halo_mpeak'] > subhalo_mpeak_cut
halo_catalog_columns = {key: halocat.halo_table[key][mask] for key in halocat.halo_table.keys()}
halo_catalog_columns.update(metadata)
halocat = UserSuppliedHaloCatalog(**halo_catalog_columns)

In [11]:
from halotools.empirical_models import PrebuiltHodModelFactory, HodModelFactory
baseline_model = PrebuiltHodModelFactory('leauthaud11', redshift=0, threshold=9.5, 
                                        prim_haloprop_key='halo_mpeak')

from segregated_nfw_phase_space import SegregatedNFWPhaseSpace
seg_nfw = SegregatedNFWPhaseSpace(prim_haloprop_key='halo_mpeak')

from halotools.empirical_models import SubhaloPhaseSpace
host_haloprop_bins = np.logspace(10.6, 15.15, 10)
d = dict(halo_id=('halo_id', 'i8'), 
        halo_mpeak=('halo_mpeak', 'f8'))
satellites_mpeak = SubhaloPhaseSpace('satellites', host_haloprop_bins,
                                        inherited_subhalo_props_dict=d)


from csmf_from_hod import StellarMassLeauthaud11HOD
csmf_model_centrals = StellarMassLeauthaud11HOD('centrals', prim_haloprop_key='halo_mpeak',
                                                threshold=baseline_model.threshold)
csmf_model_satellites = StellarMassLeauthaud11HOD('satellites', prim_haloprop_key='halo_mpeak',
                                                  threshold=baseline_model.threshold)


model_feature_calling_sequence = list(('centrals_occupation', 'satellites_occupation', 
                                       'centrals_stellar_mass', 'centrals_profile', 
                                      'satellites_mpeak', 'satellites_stellar_mass', 
                                      'satellites_profile'))

model = HodModelFactory(baseline_model_instance=baseline_model, 
                        centrals_stellar_mass=csmf_model_centrals, 
                        satellites_mpeak=satellites_mpeak, 
                        satellites_stellar_mass=csmf_model_satellites, 
                        satellites_profile=seg_nfw, 
                       model_feature_calling_sequence=model_feature_calling_sequence)


HalotoolsError: The occupation component of gal_type = ``satellites`` galaxies 
has a ``central_occupation_model`` attribute with an inconsistent 
implementation with the Leauthaud11Sats class controlling the occupation statistics of the ``centrals`` population.
If you use the ``cenocc_model`` feature, you must build a 
composite model with a self-consistent population of centrals.


In [ ]:
from csmf_from_hod import StellarMassLeauthaud11HOD
csmf_model = StellarMassLeauthaud11HOD(threshold=9)

sm = model.assign_stellar_mass(prim_haloprop=np.zeros(int(1e5))+1e12)

fig, ax = plt.subplots(1, 1)

__=ax.hist(np.log10(sm), bins=100)